# Description

The goal of this notebook is very simple: it just lists the content (traits/diseases) that belong to each cluster across all selected "best partitions". Although one would take a look at them here to check whether clusters of traits make sense, that analysis is carried out first by looking at the clustering trees (which are generated later). Then, this notebooks serves as a simple list with the content of the clusters.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

from utils import generate_result_set_name
import conf

# Settings

In [3]:
CONSENSUS_CLUSTERING_DIR = Path(
    conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering"
).resolve()

display(CONSENSUS_CLUSTERING_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering')

## Load data

In [4]:
INPUT_SUBSET = "umap"

In [5]:
INPUT_STEM = "z_score_std-projection-smultixcan-efo_partial-mashr-zscores"

In [6]:
DR_OPTIONS = {
    "n_components": 5,
    "metric": "euclidean",
    "n_neighbors": 15,
    "random_state": 0,
}

In [7]:
input_filepath = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    INPUT_SUBSET,
    generate_result_set_name(
        DR_OPTIONS, prefix=f"{INPUT_SUBSET}-{INPUT_STEM}-", suffix=".pkl"
    ),
).resolve()
display(input_filepath)

assert input_filepath.exists(), "Input file does not exist"

input_filepath_stem = input_filepath.stem
display(input_filepath_stem)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/data_transformations/umap/umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')

'umap-z_score_std-projection-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0'

In [8]:
data_umap = pd.read_pickle(input_filepath)

In [9]:
data_umap.shape

(3752, 5)

In [10]:
data_umap.head()

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
100001_raw-Food_weight,0.426554,0.670532,7.363805,1.171837,6.297295
100002_raw-Energy,-1.605179,0.815699,8.288521,0.990394,6.817351
100003_raw-Protein,-1.656178,0.788297,8.355906,1.017072,6.845651
100004_raw-Fat,-1.508325,0.802536,8.328274,1.033939,6.709319
100005_raw-Carbohydrate,-1.617872,0.812711,8.307973,1.020575,6.825944


# Load best partitions

In [11]:
input_file = Path(CONSENSUS_CLUSTERING_DIR, "best_partitions_by_k.pkl").resolve()
display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/clustering/consensus_clustering/best_partitions_by_k.pkl')

In [12]:
best_partitions = pd.read_pickle(input_file)

In [13]:
best_partitions.shape

(59, 4)

In [14]:
best_partitions.head()

,method,partition,ami_mean,selected
k,,,,
42,eac_average_coassoc_matrix,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 11, 25, 2...",0.286678,True
41,eac_average_coassoc_matrix,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 11, 25, 2...",0.286357,True
40,eac_average_coassoc_matrix,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 0, 25, 25...",0.286107,True
48,eac_average_coassoc_matrix,"[25, 25, 25, 25, 25, 25, 25, 25, 25, 6, 25, 25...",0.286006,True
17,scc_020,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.285774,True


# Analysis of clusterings

In [15]:
from IPython.display import HTML

In [16]:
def show_cluster_stats(clustering_data, selected_partition, selected_cluster):
    traits = [t for t in clustering_data[selected_partition == selected_cluster].index]
    display(len(traits))
    display(traits)

In [17]:
selected_k_values = best_partitions[best_partitions["selected"]].index.tolist()
selected_k_values.sort()
display(selected_k_values)

[12, 16, 17, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 54]

In [18]:
for k in selected_k_values:
    display(HTML(f"<h2>Partition with k={k}</h2>"))
    display(best_partitions.loc[k])

    part = best_partitions.loc[k, "partition"]
    display(part.shape)

    part_stats = pd.Series(part).value_counts()
    display(part_stats)

    for cluster_number in part_stats.index.sort_values():
        display(HTML(f"<h3>Cluster {k}.{cluster_number}</h3>"))

        cluster_traits = data_umap[part == cluster_number].index
        display(len(cluster_traits))
        display(cluster_traits)

method                              eac_average_coassoc_matrix
partition    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
ami_mean                                               0.28453
selected                                                  True
Name: 12, dtype: object

(3752,)

0     3561
6       49
1       30
11      24
3       19
4       18
2       16
9       12
7        8
5        8
8        5
10       2
dtype: int64

3561

Index(['100001_raw-Food_weight', '100002_raw-Energy', '100003_raw-Protein',
       '100004_raw-Fat', '100005_raw-Carbohydrate', '100006_raw-Saturated_fat',
       '100007_raw-Polyunsaturated_fat', '100008_raw-Total_sugars',
       '100009_raw-Englyst_dietary_fibre', '100010-Portion_size',
       ...
       'visual impairment', 'vitiligo', 'vitreous body disease',
       'vocal cord polyp', 'voice disorders',
       'wellbeing measurement AND family relationship', 'wheezing',
       'whooping cough', 'worry measurement', 'wrist fracture'],
      dtype='object', length=3561)

...

Index(['C3_SKIN-Malignant_neoplasm_of_skin',
       'C_OTHER_SKIN-Other_malignant_neoplasms_of_skin', 'C_SKIN',
       'skin neoplasm'],
      dtype='object')

14

Index(['30000_raw-White_blood_cell_leukocyte_count',
       '30130_raw-Monocyte_count', '30140_raw-Neutrophill_count',
       '30180_raw-Lymphocyte_percentage', '30190_raw-Monocyte_percentage',
       '30200_raw-Neutrophill_percentage', 'granulocyte count',
       'leukocyte count', 'lymphocyte count', 'monocyte count',
       'myeloid white cell count', 'neutrophil count',
       'sum of basophil and neutrophil counts',
       'sum of neutrophil and eosinophil counts'],
      dtype='object')

7

Index(['5984_raw-ECG_load', '5986_raw-ECG_phase_time',
       '5992-ECG_phase_duration', '5993-ECG_number_of_stages_in_a_phase',
       '6032_raw-Maximum_workload_during_fitness_test',
       '6038_raw-Number_of_trend_entries', '6039-Duration_of_fitness_test'],
      dtype='object')